## SparkSQL exercises.
### These exercises are modified from spark-sql/SparkSQL.ipynb to suit win10 environment without jdfc. All data will be loaded from csv directly.
 

In [1]:

from pyspark import SparkConf, SparkContext, SQLContext
# Create a spark configuration with 20 threads.
# This code will run locally on master
conf = (SparkConf ()
        . setMaster("local[20]")
        . setAppName("sample app for reading files")
        . set("spark.executor.memory", "2g"))

sc = SparkContext(conf=conf)

There are 2 methods which can load csv files into spark.
- load csv file by `sc.textfile` and return a spark rdd object.
- load csv file by `sqlContext.read.load` and return a spark dataframe

In [2]:
# Method 1. Load the csv into RDD format.

#csv_2_rdd = sc.textFile("big-data-3/gameclicks.csv")


In [3]:
#Method 2. Load the csv into Spark dataframe.
#print it
sqlContext = SQLContext (sc)

csv_2_df = sqlContext.read.load ("big-data-3/game-clicks.csv",
                               format='com.databricks.spark.csv',
                               header='true',
                               inferSchema='true')

csv_2_df.show()

+-------------------+-------+------+-------------+-----+------+---------+
|          timestamp|clickId|userId|userSessionId|isHit|teamId|teamLevel|
+-------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:55|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:09|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:14|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:14|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:20|     22|  1362|         5739|    0|    13|        1|
|2016-05-26 15:07:27|    107|  1071|         5939|    0|    27|        1|
|2016-05-26 15:07:30|    289|  2359|         5764|    0|    85|        1|
|2016-05-26 15:07:30|    301|  1243|         5900|    0|    86|        1|
|2016-05-26 15:07:47|    274|  1628|         5896|    0|    82|        1|
|2016-05-26 15:07:48|     66|   453|         5662|    0|    20|        1|
|2016-05-26 15:07:49|    124|  2336|  

In [4]:
csv_2_df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- clickId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- isHit: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- teamLevel: integer (nullable = true)



In [5]:
csv_2_df.count()

755806

In [6]:
csv_2_df.show(5)

+-------------------+-------+------+-------------+-----+------+---------+
|          timestamp|clickId|userId|userSessionId|isHit|teamId|teamLevel|
+-------------------+-------+------+-------------+-----+------+---------+
|2016-05-26 15:06:55|    105|  1038|         5916|    0|    25|        1|
|2016-05-26 15:07:09|    154|  1099|         5898|    0|    44|        1|
|2016-05-26 15:07:14|    229|   899|         5757|    0|    71|        1|
|2016-05-26 15:07:14|    322|  2197|         5854|    0|    99|        1|
|2016-05-26 15:07:20|     22|  1362|         5739|    0|    13|        1|
+-------------------+-------+------+-------------+-----+------+---------+
only showing top 5 rows



In [7]:
csv_2_df.select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1038|        1|
|  1099|        1|
|   899|        1|
|  2197|        1|
|  1362|        1|
+------+---------+
only showing top 5 rows



In [8]:
csv_2_df.filter(csv_2_df["teamlevel"] > 1).select("userid", "teamlevel").show(5)

+------+---------+
|userid|teamlevel|
+------+---------+
|  1513|        2|
|   868|        2|
|  1453|        2|
|  1282|        2|
|  1473|        2|
+------+---------+
only showing top 5 rows



In [9]:
csv_2_df.groupBy("ishit").count().show()

+-----+------+
|ishit| count|
+-----+------+
|    1| 83383|
|    0|672423|
+-----+------+



In [10]:
from pyspark.sql.functions import *
csv_2_df.select(mean('ishit'), sum("ishit")).show()

+------------------+----------+
|        avg(ishit)|sum(ishit)|
+------------------+----------+
|0.1103232840173272|     83383|
+------------------+----------+



Load another table from another csv using the same method.

In [11]:
csv_3_df = sqlContext.read.load ("big-data-3/ad-clicks.csv",
                               format='com.databricks.spark.csv',
                               header='true',
                               inferSchema='true')

In [12]:
csv_3_df.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- txId: integer (nullable = true)
 |-- userSessionId: integer (nullable = true)
 |-- teamId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- adId: integer (nullable = true)
 |-- adCategory: string (nullable = true)



Merge table by declaring join and common attributes

In [13]:
merge = csv_2_df.join(csv_3_df, 'userId')

In [14]:
merge.show(5)

+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+-----------+
|userId|          timestamp|clickId|userSessionId|isHit|teamId|teamLevel|          timestamp| txId|userSessionId|teamId|adId| adCategory|
+------+-------------------+-------+-------------+-----+------+---------+-------------------+-----+-------------+------+----+-----------+
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-16 10:21:01|39733|        34223|    13|   1|     sports|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-15 23:52:15|38854|        34223|    13|   3|electronics|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-15 12:23:31|37940|        34223|    13|  15|     sports|
|  1362|2016-05-26 15:07:20|     22|         5739|    0|    13|        1|2016-06-13 00:12:01|32627|        26427|    13|  14|    fashion|
|  1362|2016-05-26 15:07:20|     2